In [ ]:
# Users must check whether A2utils.py file has benn downloaded
import A2utils

In [ ]:
from Bio import AlignIO
from Bio.Align import AlignInfo
from typing import Tuple
from Bio import AlignIO
from Bio import Align
import pandas as pd
import numpy as np
import networkx as nx

In [ ]:
def load_msa(filepath: str):
    msa = AlignIO.read(filepath, "fasta")
    return AlignInfo.SummaryInfo(msa.alignment)

def load_sequences(filepath: str):
    from Bio import SeqIO
    seqs = []
    for seq in SeqIO.parse(filepath, "fasta"):
        seqs.append((seq.name, str(seq.seq)))
    return seqs

In [11]:
# We implement the UPGMA for phylo tree construction
# We assign sequence name as label under each node, and each node is an unique integers.
# Edge length is added under len attribute.
def grab_leaves(tree, internal_node, leaf_nodes_lst):
    descendants_inter = set(nx.descendants(tree, internal_node))
    leaves_for_internal_nodes = list(descendants_inter & set(leaf_nodes_lst))
    return leaves_for_internal_nodes

def build_evo_tree(sequences: Tuple[str, str], msa: AlignInfo.SummaryInfo) -> nx.DiGraph:
    """
    Performs evolutionary tree building using input sequences and/or an input msa.
    """
    # YOUR CODE HERE
    # calculate the distance matrix
    aligner = Align.PairwiseAligner()
    try:
        aligner.substitution_matrix = Bio.Align.substitution_matrices.load('BLOSUM80')
    except:
        aligner.substitution_matrix = Bio.Align.substitution_matrices.load('blosum80')
    n = len(seqs)
    labels = [seqs[i][0] for i in range(n)]
    dist_mat = pd.DataFrame(np.full(shape = (n, n), fill_value = ''))
    for i in range(n):
        for j in range(n):
            if i == j:
                dist_mat.at[i, j] = np.nan  # if idx is the same, put nan first
            else:
                if dist_mat.loc[i, j] != '':
                    dist_mat.at[i, j] = dist_mat.loc[j, i]  # if we've calculated the distance for that pair, just replace its value (no need to calculate again)
                else:
                    costs = aligner.align(msa.alignment[i], msa.alignment[j]).score
                    dist_mat.at[i, j] = costs

    max_score = dist_mat.max().max()
    dist_mat = 1 - dist_mat/max_score  # score normalization
    dist_mat = dist_mat.fillna(0)      # fill the nan as zero

    # First adding all leaf nodes inside the phylogenic tree
    leaf_dct = dict(zip([i for i in range(len(labels))], labels)) # create a dict for leaf nodes, the structure is number : strain_name
    tree = nx.DiGraph()
    for i in range(len(leaf_dct)):
        tree.add_node(i, label = leaf_dct[i])  # First ingest all the nodes

    # inter_name = [] # internal nodes, same as leaf_dct; number : strain_name
    innode_name = n  # set the idx for internal node

    # start to build the tree
    while n > 1:
        # find the minimum distance inside the tree
        min_dist = dist_mat[dist_mat != 0].min().min()  # zero distance means two selected nodes from col and row are the same, we don't want to pick it up.
        cols = list(dist_mat.columns)
        # find the first pair with the minimum distance
        min_i, min_j = cols[np.where(dist_mat == min_dist)[0][0]],  cols[np.where(dist_mat == min_dist)[1][0]]
        
        # two nodes are leaf nodes
        if leaf_dct.get(min_i) and leaf_dct.get(min_j):
            tree.add_edge(innode_name, min_i, weight = min_dist/2)
            tree.add_edge(innode_name, min_j, weight = min_dist/2)
            # inter_name.append(innode_name)
            node1 = min_i; node2 = min_j

        # other cases. Writing else directly is because I want to make it clear.
        else:
            # one in leaf, and the other is internal
            if leaf_dct.get(min_i) is None and leaf_dct.get(min_j):
                leaf_idx = min_j; inter_idx = min_i
                tree.add_edge(innode_name, leaf_idx, weight = min_dist/2)
                descendants_inter = set(nx.descendants(tree, inter_idx)); leaves_for_inter = list(descendants_inter & set(leaf_dct.keys()))
                tree.add_edge(innode_name, inter_idx, weight = min_dist/2 - nx.shortest_path_length(G = tree, source = inter_idx, target = leaves_for_inter[0]) )
                node1 = leaf_idx; node2 = inter_idx

            # one in leaf, and the other is internal
            elif leaf_dct.get(min_j) is None and leaf_dct.get(min_i):
                leaf_idx = min_i; inter_idx = min_j
                tree.add_edge(innode_name, leaf_idx, weight = min_dist/2)
                descendants_inter = set(nx.descendants(tree, inter_idx)); leaves_for_inter = list(descendants_inter & set(leaf_dct.keys()))
                tree.add_edge(innode_name, inter_idx, weight = min_dist/2 - nx.shortest_path_length(G = tree, source = inter_idx, target = leaves_for_inter[0]) )
                node1 = leaf_idx; node2 = inter_idx

          # Two are all internal nodes
            else:
                leaves_for_inter1 = grab_leaves(tree, min_i, list(leaf_dct.keys()))
                leaves_for_inter2 = grab_leaves(tree, min_j, list(leaf_dct.keys()))
                tree.add_edge(innode_name, min_i, weight = min_dist/2 - nx.shortest_path_length(G = tree, source = min_i, target = leaves_for_inter1[0]) )
                tree.add_edge(innode_name, min_j, weight = min_dist/2 - nx.shortest_path_length(G = tree, source = min_j, target = leaves_for_inter2[0]) )
                node1 = min_i; node2 = min_j

        # update distance matrix
        new_distance = []
        for k in dist_mat.index:
            # calculate the distance of nodes left in the distance matrix
            if k != node1 and k != node2:
                new_dist = (dist_mat.loc[node1, k] + dist_mat.loc[node2, k])/2
                new_distance.append(new_dist)
        
        # We drop two nodes selected in this round and append the newly-formed internal node inside the distance matrix
        dist_mat = dist_mat.drop([node1, node2], axis = 1).drop([node1, node2], axis = 0)
        dist_mat[innode_name] = 0; dist_mat.loc[innode_name] = 0
        dist_mat[innode_name] = new_distance + [0]; dist_mat.loc[innode_name] = new_distance + [0]

        n = len(dist_mat); innode_name += 1  # update n and update the internal node index

    return tree

In [ ]:
# We can load the MSA.fasta and fasta file into the function.

seqs = load_sequences(EBOLA_SEQS_FILEPATH)
msa = load_msa(EBOLA_MSA_FILEPATH)
T = build_evo_tree(seqs, msa)
A2utils.draw_evo_tree(T)